In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from experiments.atari import EXPERIMENTED_GAME

experiments = ["ut30_uh6000"]
baselines = ['IQN', 'Rainbow', 'REM', 'DQN (Adam)', 'C51', 'DQN (Nature)']
baselines_performance_profile = ['IQN', 'REM', 'DQN (Adam)']
games = EXPERIMENTED_GAME
ks = [5]
seeds = [11, 21]
plot_std = True
selected_epochs = np.array([1, 10, 25, 50, 75, 100, 125, 150, 175, 200]) - 1
taus = np.linspace(0.0, 8.0, 81)

### Extract data

In [ ]:
from idqn.utils.baselines_scores import get_baselines_scores

idqn_scores = {}
for experiment in experiments:
    for k in ks:
        idqn_scores[f"{experiment}_{k}"] = {}
        for game in games:
            idqn_scores[f"{experiment}_{k}"][game] = np.zeros((200, len(seeds))) * np.nan
            for idx_seed, seed in enumerate(seeds):
                idqn_scores[f"{experiment}_{k}"][game][:, idx_seed] = np.load(f"figures/{experiment}/{game}/iDQN/{k}_J_{seed}.npy")

baselines_scores = get_baselines_scores(baselines, games)

In [ ]:
import matplotlib.pyplot as plt
from idqn.utils.process_scores import compute_iqm_and_confidence_interval

plt.rc("font", size=15)
plt.rc("lines", linewidth=3)


for experiment in experiments:
    for k in ks:

        iqms, iqms_confidence_interval = compute_iqm_and_confidence_interval(idqn_scores[f"{experiment}_{k}"], selected_epochs)
        plt.plot(selected_epochs + 1, iqms, label=f"iDQN {experiment} K={k}")
        if plot_std:
            plt.fill_between(selected_epochs + 1, iqms_confidence_interval[0, :], iqms_confidence_interval[1, :], alpha=0.3)

for baseline in baselines:
    iqms, iqms_confidence_interval = compute_iqm_and_confidence_interval(baselines_scores[baseline], selected_epochs)
    plt.plot(selected_epochs + 1, iqms, label=baseline)
    if plot_std:
        plt.fill_between(selected_epochs + 1, iqms_confidence_interval[0, :], iqms_confidence_interval[1, :], alpha=0.3)

plt.grid()
plt.xlabel("Number of Frames (in millions)")
plt.ylabel("IQM Human Normalized Score")
if len(games) == 1 and len(experiments) > 0:
    plt.legend(bbox_to_anchor=[1, 1])
    plt.title(games[0])
    _ = plt.savefig(f"figures/{experiments[0]}/{games[0]}/J.pdf", bbox_inches='tight')
elif len(experiments) > 0:
    plt.legend(ncols=len(baselines) + len(experiments) * len(ks), bbox_to_anchor=[1, 1])
    _ = plt.savefig(f"figures/{experiments[0]}/J.pdf", bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
from idqn.utils.process_scores import compute_performance_profile_and_confidence_interval

plt.rc("font", size=15)
plt.rc("lines", linewidth=3)


if len(games) > 1:
    for experiment in experiments:
        for k in ks:

            performance_profile, performance_profile_confidence_interval = compute_performance_profile_and_confidence_interval(idqn_scores[f"{experiment}_{k}"], taus)
            plt.plot(taus, performance_profile, label=f"iDQN {experiment} K={k}")
            if plot_std:
                plt.fill_between(taus, performance_profile_confidence_interval[0, :], performance_profile_confidence_interval[1, :], alpha=0.3)

    for baseline in baselines_performance_profile:
        performance_profile, performance_profile_confidence_interval = compute_performance_profile_and_confidence_interval(baselines_scores[baseline], taus)
        plt.plot(taus, performance_profile, label=baseline)
        if plot_std:
            plt.fill_between(taus, performance_profile_confidence_interval[0, :], performance_profile_confidence_interval[1, :], alpha=0.3)

    plt.grid()
    plt.xlabel(r"Human Normalized Score $(\tau)$")
    plt.ylabel(r"Fraction of runs with score $> \tau$")
    if len(games) == 1 and len(experiments) > 0:
        plt.legend(bbox_to_anchor=[1, 1])
        plt.title(games[0])
        _ = plt.savefig(f"figures/{experiments[0]}/{games[0]}/P.pdf", bbox_inches='tight')
    elif len(experiments) > 0:
        plt.legend(ncols=len(baselines) + len(experiments) * len(ks), bbox_to_anchor=[1, 1])
        _ = plt.savefig(f"figures/{experiments[0]}/P.pdf", bbox_inches='tight')